In [2]:
# import necessary dependencies
import numpy as np 
import pandas as pd 
import altair as alt
from functools import reduce # to merge datasets

In [3]:
# 
CH4_capita = pd.read_csv("ghg-emissions-ch4-capita.csv")
CH4_gdp = pd.read_csv("ghg-emissions-ch4-gdp.csv")
CH4_total = pd.read_csv("ghg-emissions-ch4-total.csv")

#
CO2_capita = pd.read_csv("ghg-emissions-co2-capita.csv")
CO2_gdp = pd.read_csv("ghg-emissions-co2-gdp.csv")
CO2_total = pd.read_csv("ghg-emissions-co2-total.csv")

#
fgas_capita = pd.read_csv("ghg-emissions-fgas-capita.csv")
fgas_gdp = pd.read_csv("ghg-emissions-fgas-gdp.csv")
fgas_total = pd.read_csv("ghg-emissions-fgas-total.csv")

#
N20_capita = pd.read_csv("ghg-emissions-n2o-capita.csv")
N20_gdp = pd.read_csv("ghg-emissions-n2o-gdp.csv")
N20_total = pd.read_csv("ghg-emissions-n2o-total.csv")

In [4]:
#
CH4_capita_melt = pd.melt(CH4_capita, id_vars = ["iso", "Country/Region", "unit"], var_name = "Year", value_name = "CH4pC").drop(columns = "unit")
CH4_gdp_melt = pd.melt(CH4_gdp, id_vars = ["iso", "Country/Region", "unit"], var_name = "Year", value_name = "CH4pGDP").drop(columns = "unit")
CH4_total_melt = pd.melt(CH4_total, id_vars = ["iso", "Country/Region", "unit"], var_name = "Year", value_name = "CH4TOT").drop(columns = "unit")

#
CO2_capita_melt = pd.melt(CO2_capita, id_vars = ["iso", "Country/Region", "unit"], var_name = "Year", value_name = "CO2pC").drop(columns = "unit")
CO2_gdp_melt = pd.melt(CO2_gdp, id_vars = ["iso", "Country/Region", "unit"], var_name = "Year", value_name = "CO2pGDP").drop(columns = "unit")
CO2_total_melt = pd.melt(CO2_total, id_vars = ["iso", "Country/Region", "unit"], var_name = "Year", value_name = "CO2TOT").drop(columns = "unit")

#
fgas_capita_melt = pd.melt(fgas_capita, id_vars = ["iso", "Country/Region", "unit"], var_name = "Year", value_name = "FGaspC").drop(columns = "unit")
fgas_gdp_melt = pd.melt(fgas_gdp, id_vars = ["iso", "Country/Region", "unit"], var_name = "Year", value_name = "FGaspGDP").drop(columns = "unit")
fgas_total_melt = pd.melt(fgas_total, id_vars = ["iso", "Country/Region", "unit"], var_name = "Year", value_name = "FGasTOT").drop(columns = "unit")

#
N20_capita_melt = pd.melt(N20_capita, id_vars = ["iso", "Country/Region", "unit"], var_name = "Year", value_name = "N20pC").drop(columns = "unit")
N20_gdp_melt = pd.melt(N20_gdp, id_vars = ["iso", "Country/Region", "unit"], var_name = "Year", value_name = "N20pGDP").drop(columns = "unit")
N20_total_melt = pd.melt(N20_total, id_vars = ["iso", "Country/Region", "unit"], var_name = "Year", value_name = "N20TOT").drop(columns = "unit")

#
N20_total_melt

,iso,Country/Region,Year,N20TOT
0,USA.TX,Texas,1990,55.12
1,USA.IA,Iowa,1990,25.48
2,USA.KS,Kansas,1990,20.84
3,USA.CA,California,1990,18.62
4,USA.IL,Illinois,1990,24.82
...,...,...,...,...
1576,USA.DE,Delaware,2020,0.48
1577,USA.NH,New Hampshire,2020,0.35
1578,USA.AK,Alaska,2020,0.22
1579,USA.RI,Rhode Island,2020,0.20


In [18]:
#
dfs = [CH4_capita_melt, CH4_gdp_melt, CH4_total_melt, CO2_capita_melt, CO2_gdp_melt, CO2_total_melt, fgas_capita_melt, fgas_gdp_melt, fgas_total_melt, N20_capita_melt, N20_gdp_melt, N20_total_melt]

#
ghg = reduce(lambda left, right: pd.merge(left, right, on = ["iso", "Country/Region", "Year"], how = "left"), dfs)

#
ghg.head()

,iso,Country/Region,Year,CH4pC,CH4pGDP,CH4TOT,CO2pC,CO2pGDP,CO2TOT,FGaspC,FGaspGDP,FGasTOT,N20pC,N20pGDP,N20TOT
0,USA.TX,Texas,1990,6.33,246.87,107.96,32.16,1254.40,548.54,0.26,10.31,4.51,3.23,126.05,55.12
1,USA.CA,California,1990,2.15,71.05,64.37,10.42,344.40,312.06,0.10,3.19,2.89,0.62,20.55,18.62
2,USA.PA,Pennsylvania,1990,3.13,128.85,37.23,18.61,766.75,221.56,0.05,2.24,0.65,0.70,29.02,8.39
3,USA.OK,Oklahoma,1990,10.87,510.14,34.22,25.32,1188.46,79.73,0.20,9.62,0.65,3.71,174.05,11.68
4,USA.WV,West Virginia,1990,20.10,1149.17,36.03,45.88,2623.43,82.24,0.63,35.78,1.12,1.18,67.58,2.12


In [19]:
# rename states column
ghg = ghg.rename(columns = {"Country/Region": "State"})

# remove USA. from iso
ghg["iso"] = ghg["iso"].str.extract(r'USA\.(.*)')

# Assign regions
def get_region(state):
    if state in ['AK', 'CA', 'CO', 'HI', 'ID', 'MT', 'NV', 'OR', 'UT', 'WA', 'WY']:
        return 'West'
    elif state in ['AZ', 'NM', 'OK', 'TX']:
        return 'Soutwest'
    elif state in ['IA', 'IL', 'IN', 'KS', 'MI', 'MN', 'MO', 'ND', 'NE', 'OH', 'SD', 'WI']:
        return 'Midwest'
    elif state in ['AL', 'AR', 'DC', 'DE', 'FL', 'GA', 'KY', 'LA', 'MD', 'MS', 'NC', 'SC', 'TN', 'VA', 'WV']:
        return 'Southeast'
    elif state in ['CT', 'MA', 'ME', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT']:
        return 'Northeast'
    else:
        return 'Other'

ghg["Region"] = ghg["iso"].apply(get_region)
ghg

,iso,State,Year,CH4pC,CH4pGDP,CH4TOT,CO2pC,CO2pGDP,CO2TOT,FGaspC,FGaspGDP,FGasTOT,N20pC,N20pGDP,N20TOT,Region
0,TX,Texas,1990,6.33,246.87,107.96,32.16,1254.40,548.54,0.26,10.31,4.51,3.23,126.05,55.12,Soutwest
1,CA,California,1990,2.15,71.05,64.37,10.42,344.40,312.06,0.10,3.19,2.89,0.62,20.55,18.62,West
2,PA,Pennsylvania,1990,3.13,128.85,37.23,18.61,766.75,221.56,0.05,2.24,0.65,0.70,29.02,8.39,Northeast
3,OK,Oklahoma,1990,10.87,510.14,34.22,25.32,1188.46,79.73,0.20,9.62,0.65,3.71,174.05,11.68,Soutwest
4,WV,West Virginia,1990,20.10,1149.17,36.03,45.88,2623.43,82.24,0.63,35.78,1.12,1.18,67.58,2.12,Southeast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1576,NH,New Hampshire,2020,0.68,12.34,0.94,4.57,83.04,6.31,0.40,7.25,0.55,0.25,4.60,0.35,Northeast
1577,DE,Delaware,2020,0.80,12.70,0.79,11.93,190.11,11.84,0.40,6.35,0.40,0.48,7.63,0.48,Southeast
1578,HI,Hawaii,2020,0.46,9.55,0.67,9.74,201.53,14.13,0.52,10.75,0.75,0.40,8.25,0.58,West
1579,RI,Rhode Island,2020,0.27,5.73,0.30,8.27,175.99,9.07,0.38,8.16,0.42,0.18,3.92,0.20,Northeast


In [21]:
ghg.isna().sum()

iso         0
State       0
Year        0
CH4pC       0
CH4pGDP     0
CH4TOT      0
CO2pC       0
CO2pGDP     0
CO2TOT      0
FGaspC      0
FGaspGDP    0
FGasTOT     0
N20pC       0
N20pGDP     0
N20TOT      0
Region      0
dtype: int64

In [37]:
fig1 = alt.Chart(ghg).transform_filter(
    alt.FieldEqualPredicate(field='Region', equal='West')
).encode(
    x = alt.X("Year:O"),
    y = alt.Y("CH4TOT:Q"),
    color = alt.Color("Region:N")
).mark_line(point=True)

fig1

alt.Chart(...)

Trying to decide if we should use the mean or median because the data is right skewed.


In [38]:
fig2 = alt.Chart(ghg).transform_filter(
    alt.FieldEqualPredicate(field='Region', equal='West')
).encode(
    x = alt.X("Year:O"),
    y = alt.Y("mean(CH4TOT):Q"),
    color = alt.Color("Region:N")
).mark_line(point=True)

fig3 = alt.Chart(ghg).transform_filter(
    alt.FieldEqualPredicate(field='Region', equal='West')
).encode(
    x = alt.X("Year:O"),
    y = alt.Y("median(CH4TOT):Q"),
    color = alt.Color("Region:N")
).mark_line(point=True)

fig2 | fig3

alt.HConcatChart(...)